# Desafio - Aula 2

### Nome: Jonny Silva Canatto
### RM: 332439

In [1]:
#Realizando todos os imports necessários
import pandas as pd
import numpy as np
import pandas as pd
import nltk
import spacy
import re

from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import CountVectorizer
from collections import Counter
from nltk.corpus import floresta
from nltk.stem.rslp import RSLPStemmer

nlp = spacy.load('pt_core_news_sm')
nltk.download('floresta')
stops = nltk.corpus.stopwords.words('portuguese')

[nltk_data] Downloading package floresta to
[nltk_data]     C:\Users\jonny\AppData\Roaming\nltk_data...
[nltk_data]   Package floresta is already up-to-date!


In [2]:
df = pd.read_csv("https://s3.amazonaws.com/automl-example/produtos.csv",
                 delimiter=";", encoding='utf-8')

### 1) utilizando o df acima carregado, faça:

   ### -> Elimine linhas com valores nulos
   ### -> Adicione uma nova coluna chamada texto, formada pela composição das colunas nome e descrição
   ### -> Quantos Unigramas existem antes e depois de remover stopwords
   ### -> Quantos Bigramas existem antes e depois de remover stopwords
   ### -> Quantos Trigramas existem antes e depois de remover stopwords
   ### -> Quantos verbos e Adverbios existem na nova coluna (utilize: from collections import Counter)
   ### -> Quantos unigramas existem após aplicar Stemmer (utilize rslp)

In [3]:
#tratamento dos dados

#Removendo duplicados
print('Removendo duplicados')
print('Tamanho total do data frame:', len(df))
df = df.drop_duplicates()
print('Após remoção de duplicados:', len(df))

print('')

#removendo nulos
print('Removendo nulos')
print('Tamanho total com nulos:', len(df))
df = df.dropna()
print('Tamanho sem nulos:', len(df))

Removendo duplicados
Tamanho total do data frame: 4080
Após remoção de duplicados: 3789

Removendo nulos
Tamanho total com nulos: 3789
Tamanho sem nulos: 2646


In [4]:
#Adicione uma nova coluna chamada texto, formada pela composição das colunas nome e descrição
df['texto'] = df['nome'] + ' ' + df['descricao']

In [5]:
#Quantos Unigramas existem antes e depois de remover stopwords
vect = CountVectorizer(ngram_range=(1,1))
vect.fit(df['texto'])
text_vect = vect.transform(df['texto'])
print('Unigramas com stop words: ', text_vect.shape[1])

vect1 = CountVectorizer(ngram_range=(1,1), stop_words=stops)
vect1.fit(df['texto'])
text_vect1 = vect1.transform(df['texto'])
print('Unigramas sem stop words: ', text_vect1.shape[1])

Unigramas com stop words:  35466
Unigramas sem stop words:  35310


In [6]:
#Quantos Bigramas existem antes e depois de remover stopwords
vect = CountVectorizer(ngram_range=(2,2))
vect.fit(df['texto'])
text_vect = vect.transform(df['texto'])
print('Bigramas com stop words: ', text_vect.shape[1])

vect1 = CountVectorizer(ngram_range=(2,2), stop_words=stops)
vect1.fit(df['texto'])
text_vect1 = vect1.transform(df['texto'])
print('Bigramas sem stop words: ', text_vect1.shape[1])

Bigramas com stop words:  159553
Bigramas sem stop words:  145409


In [7]:
#Quantos Trigramas existem antes e depois de remover stopwords
vect = CountVectorizer(ngram_range=(3,3))
vect.fit(df['texto'])
text_vect = vect.transform(df['texto'])
print('Trigramas com stop words: ', text_vect.shape[1])

vect1 = CountVectorizer(ngram_range=(3,3), stop_words=stops)
vect1.fit(df['texto'])
text_vect1 = vect1.transform(df['texto'])
print('Trigramas sem stop words: ', text_vect1.shape[1])

Trigramas com stop words:  228162
Trigramas sem stop words:  177869


In [8]:
#Quantos verbos e Adverbios existem na nova coluna (utilize: from collections import Counter)
# VERB
# ADV

contador = Counter()
verbs = 0
advs = 0

for index, row in df.iterrows():
    texto = nlp(row['texto'])
    contador = [(token.pos_) for token in texto]
    if('VERB' in contador):
        verbs += contador.index('VERB')
    if('ADV' in contador):
        advs += contador.index('ADV')
        
print('Qtd de Verbos na nova coluna:', verbs)
print('Qtd de Adverbios na nova coluna:', advs)

Qtd de Verbos na nova coluna: 58842
Qtd de Adverbios na nova coluna: 102352


In [9]:
#Quantos unigramas existem após aplicar Stemmer (utilize rslp)
#Unigramas sem stop words:  35310

rslp = RSLPStemmer()

stem = [rslp.stem(x) for x in word_tokenize(df['texto'].to_string())]
df_stem = pd.DataFrame({'texto':stem})

vect = CountVectorizer(ngram_range=(1,1), stop_words=stops)
vect.fit(df_stem.texto)
text_vect = vect.transform(df_stem.texto)
print('Unigramas após aplicar Stemmer sem stop words: ', text_vect.shape[1])

Unigramas após aplicar Stemmer sem stop words:  5766


### 2) crie um tagger baseado em expressões regulares:
   ### -> crie uma variável que amazenará uma tupla (expressão, tag)
   ### -> o tagger deverá capturar gerúndio (ando, endo, indo), plurais e números cardinais
   ### -> utilize nltk.RegexpTagger(variável) para carregar seu tagger
   ### -> apresente uma frase teste para cada tipo de expressão

In [74]:
#crie uma variável que amazenará uma tupla (expressão, tag)

tupla = (['.*ndo$','GRN'],['[U u]m','CARD'],['[D d]ois','CARD'],['[T t]res','CARD'],['[Q q]uatro','CARD'],['[C c]inco','CARD'],
         ['[S s]eis','CARD'],['[S s]ete','CARD'],['[O o]ito','CARD'],['[N n]ove','CARD'],['[D d]ez','CARD'], ['.*s$','PLUR'])

print(type(tupla))

#o tagger deverá capturar gerúndio (ando, endo, indo), plurais e números cardinais
tagger = nltk.RegexpTagger(tupla)
print(type(tupla))

string_gerundio = 'Estavamos andando na rua'
string_plural = 'Varias pessoas foram presas'
string_cardinal = 'Tirei dez na materia de NLP'

print(' ')

#Gerundio
print(tagger.tag([palavra for palavra in string_gerundio.split(' ')]))

#Plural
print(tagger.tag([palavra for palavra in string_plural.split(' ')]))

#Cardinal
print(tagger.tag([palavra for palavra in string_cardinal.split(' ')]))

<class 'tuple'>
<class 'tuple'>
 
[('Estavamos', 'PLUR'), ('andando', 'GRN'), ('na', None), ('rua', None)]
[('Varias', 'PLUR'), ('pessoas', 'PLUR'), ('foram', None), ('presas', 'PLUR')]
[('Tirei', None), ('dez', 'CARD'), ('na', None), ('materia', None), ('de', None), ('NLP', None)]
